# Export Google Scholar citations as BibTeX

First, we install and import the required libraries.

In [1]:
!pip install scholarly pybtex

Initialize scholarly and set up a scraper with rotating proxies to avoid blocking

In [2]:
from scholarly import scholarly, ProxyGenerator
pg = ProxyGenerator()
pg.FreeProxies()
scholarly.use_proxy(pg)

Fetch author data and verify we got the correct profile

In [3]:
author = scholarly.search_author_id('zQgGTr4AAAAJ')
author_data = scholarly.fill(author)
print(f"Found author: {author_data['name']}")

Found author: Dr. Josefine Umlauft


Retrieve all publications and their BibTeX entries

In [4]:
bibtex_entries = []
for pub in scholarly.fill(author_data, sections=['publications'])['publications']:
    try:
        filled_pub = scholarly.fill(pub)
        if 'bib' in filled_pub:
            # Convert publication data to BibTeX string format
            bib_data = filled_pub['bib']
            bib_str = f"@{bib_data.get('ENTRYTYPE', 'article')}{{{bib_data.get('ID', 'unknown')},\n"
            for key, value in bib_data.items():
                if key not in ['ENTRYTYPE', 'ID']:
                    bib_str += f"    {key} = {{{value}}},\n"
            bib_str += "}"
            bibtex_entries.append(bib_str)
    except Exception as e:
        print(f"Error processing publication: {e}")

Save BibTeX entries to file

In [5]:
with open('scholar_publications.bib', 'w', encoding='utf-8') as f:
    f.write('\n\n'.join(bibtex_entries))